In [ ]:
from abc import ABC, abstractmethod
from collections import Counter, defaultdict
from dataclasses import dataclass
from enum import Enum
import itertools
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import random
from typing import Dict, List, Optional, Set, Tuple, Union

In [ ]:
from basic_utils import (
    ALL_ROLL_TUPLES,
    Box,
    BoxCategories,
    RollAction,
    RollValues,
    ScoreAction,
    ScoreCard,
    roll_first,
    remove_dice,
    roll_again,
    GameState,
    ROLL_TUPLES_BY_BOX,
)
from agents import Agent, EpsilonGreedyAgent, GreedyAgent, RandomAgent
from tabulate_best_actions import all_expected_scores_table, expected_scores_table

A state in the game consists of a `ScoreCard` (score card state), `RollValues` (values of dice showing on the table), and `rolls_completed` from 1 to 3 within the turn. The `ScoreCard` contains all needed information from the previous turns. The `RollValues` just contains the values of the five dice that have been rolled at a given point.

`GameState` contains all three of these objects. It provides `possible_score_actions`, which gives the scores possible with a given set of dice values and score card state. I frame these as actions because at any time the player can choose to end their turn and score with one of these values. For convenience, they are sorted in descending order by score. `GameState` also provides `possible_actions`, which includes roll actions in addition to the `possible_score_actions`. The `re_roll` method takes dice that are specified by value and rolls again. Finally, `GameState` provides an `update_score` method, which updates the scorecard given a choice of box.

The current score can be accessed at any time by calling the `score` method of the `GameState`'s `scorecard`.

In [ ]:
random_agent_scores = RandomAgent().play_games(n_games=10_000, histogram_bins=20)

The mean score for a `RandomAgent` over 10,000 games is less than 50 - that's pretty bad! There is a long tail on the right, with a few scores above 100 being achieved.

In this first pass at implementing the gameplay actions and scoring, I focused on development speed and cleanliness of the design. I deliberatly did not spend time optimizing for runtime speed, and it shows here, as it took several minutes to simulate 10,000 one-player games. I still think this is OK for now; I can optimize later once I start training some deep RL agents.

Let's try a `GreedyAgent` instead.

In [ ]:
greedy_agent_scores = GreedyAgent().play_games(n_games=1000, histogram_bins=20)

In [ ]:
sum([1 for s in greedy_agent_scores if s > 150]) / len(greedy_agent_scores)

The `GreedyAgent` does substantially better: it achieves mean score above 100, in about 5% of games it scores better than 150, and in a few games it even gets close to 200. 

Still, the performance is not great compared to typcial human gameplay. I'm not a terribly skilled player, and my own threshold for a good game is a score around 200.

It's pretty clear that a naive $\epsilon$-greedy agent won't do better. Since dice have no memory, there's no advantage of sometimes taking a `RollAction` instead of the best `ScoreAction`; to do better, we'd have to pick our `RollAction` strategically, not at random. Trying out the naive agent confirms that it's not a better approach:

In [ ]:
epsilon_greedy_agent_scores = EpsilonGreedyAgent(epsilon=0.5).play_games(n_games=1000)

Next I'll try to come up with a reasonably good strategy, similar to the one I try to follow when playing in the real world. Roughly speaking, given the five dice values after the first roll I pick an unused box to "go for" and then roll the dice that maximize the probability of hitting the box (or scoring as high as possible in the box, for the upper section) on this turn. For now let's set aside the question of how to pick which box to go for, and focus on figuring out which dice to roll to hit the box.

Given `roll_values` and `box`, I need to figure out the `roll_action` for that box, which I define as the one that maximizes the expected score *from that box* when `roll_action` is taken. Note that I am not computing the total expected score from the `roll_action` on `roll_values`, just the expected score that comes from the particular box. The idea is that my agent will simply look at the dice and either take a `ScoreAction` or pick a box to "roll for". The advantage of this is that it places a smaller limit on the number of possible actions than considering all possible roll actions. However, there may be cases where this is suboptimal: say `roll_action_1` is the best to hit `SmallStraight` and `roll_action_2` is the best to hit `FullHouse`, but there is some other action `roll_action_3` that has higher sum of expected scores for `SmallStraight` and `FullHouse`. I'll neglect such cases for this agent.

Figuring out the best `RollAction` for a small or large straight is not as straightforward (ha) as for the other categories. Given particular `RollValues` it's usually pretty easy to see what to roll, but the casework here could make the function implementation a bit messy. Instead of attempting to write out all the cases, I just precomputed a map from the roll values to the best roll action to take.

In [ ]:
expected_scores_table().loc[(1, 1, 1, 2, 2), :].loc[Box.FullHouse.name]["dice_values_to_roll"]

In [ ]:
def best_roll_action_for_box_with_score(roll_values: RollValues, box: Box) -> Tuple[Optional[RollAction], float]:
    """
    Given a set of dice values and a box, returns the dice to roll in order to maximize
    the expected value of the score in the box, and also gives the expecte score. If
    rolling no dice gives the best score, then None is returned as the roll action.
    """
    row = expected_scores_table().loc[roll_values.values, :].loc[box.name]
    roll_action_tuple, score = row["dice_values_to_roll"], row["expected_score"]
    roll_action = RollAction(*roll_action_tuple) if len(roll_action_tuple) != 0 else None
    return roll_action, score

def best_roll_action_for_box(roll_values: RollValues, box: Box) -> Optional[RollAction]:
    """
    Given a set of dice values and a box, returns the dice to roll in order to maximize
    the expected value of the score in the box. If rolling no dice gives the best score,
    then None is returned.
    """
    return best_roll_action_for_box_with_score(roll_values, box)[0]

def best_action_by_box_with_score(roll_values: RollValues, allowed_boxes: Set[Box]) -> Dict[Box, Tuple[Union[RollAction, ScoreAction], float]]:
    result = {}
    for box in allowed_boxes:
        best_roll_action, score = best_roll_action_for_box_with_score(roll_values, box)
        best_action = best_roll_action if best_roll_action is not None else ScoreAction(roll_values.score_from_box(box), roll_values.values, box)
        result[box] = (best_action, score)
    return result

def greedy_best_action(roll_values: RollValues, allowed_boxes: Set[Box]) -> Union[RollAction, ScoreAction]:
    return max(best_action_by_box_with_score(roll_values, allowed_boxes).values(), key=lambda x: x[1])[0]

In [ ]:
d = best_action_by_box_with_score(RollValues(1, 1, 2, 2, 2), {box for box in Box})
d

In [ ]:
game_state = GameState()
game_state.start_turn()

In [ ]:
action = greedy_best_action(game_state.roll_values, game_state.scorecard.unused_boxes)
print(action)
game_state.take_action(action)

In [ ]:
class GreedyExpectedScoresAgent(Agent):
    """
    After each of the first two rolls in a turn, this agent chooses the action that
    has the highest expected score for some box. After the third roll, it simply
    selects the ScoreAction with the highest score (if a ScoreAction has not already
    been taken on that turn).

    Note that this agent does not necessarily take the action with the highest expected
    score, just with the highest expected score for some box. You should think of this
    agent as choosing a "box action" from the available boxes: it looks at the boxes,
    determines the action that maximizes the expected score from each, and then out of
    those actions picks the max.
    """

    def choose_action(self, game_state: GameState) -> Union[RollAction, ScoreAction]:
        if game_state.rolls_completed < 3:
            return greedy_best_action(game_state.roll_values, game_state.scorecard.unused_boxes)
        else:
            return game_state.sorted_possible_score_actions[0]

In [ ]:
GreedyExpectedScoresAgent(narrate=True).play_single_game()

In [ ]:
greedy_expected_scores_agent_scores = GreedyExpectedScoresAgent().play_games(n_games=1_000)

In [ ]:
greedy_expected_scores_agent_scores_2 = GreedyExpectedScoresAgent().play_games(n_games=10_000)

In [ ]:
len([x for x in greedy_expected_scores_agent_scores_2 if x > 250]) / len(greedy_expected_scores_agent_scores_2)

In [ ]:
np.median(greedy_expected_scores_agent_scores_2)

This agent does much better. In 10,000 games, it got a median score of 197 and mean score of 204, and it scored over 250 more than 10% of the time. Here's a histogram of the results:

![Results](greedy_expected_scores_agent_10_000_games_hist.png)

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(greedy_expected_scores_agent_scores_2, bins=50)
plt.xlabel("Score")
plt.ylabel("Number of games")
plt.title("Scores from 10000 Yahtzee games with GreedyExpectedScoresAgent")
plt.show()